<a href="https://colab.research.google.com/github/110805/Retinopathy_detection/blob/master/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/110805/Retinopathy_detection.git
%cd Retinopathy_detection/

Cloning into 'Retinopathy_detection'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 35153 (delta 9), reused 6 (delta 3), pack-reused 35135
Receiving objects: 100% (35153/35153), 556.73 MiB | 32.03 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (35133/35133), done.
/content/Retinopathy_detection


In [0]:
from dataloader import RetinopathyLoader
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models
import matplotlib.pyplot as plt
import copy

# Hyperparameter setting
batch_size = 4
learning_rate = 1e-3
epochs_18 = 10
epochs_50 = 5
momentum = 0.9
weight_decay = 5e-4
criterion = nn.CrossEntropyLoss()

train_data = RetinopathyLoader(root='/content/Retinopathy_detection/data/', mode='train')
test_data = RetinopathyLoader(root='/content/Retinopathy_detection/data/', mode='test')
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

class ResNet(nn.Module):
    def __init__(self, pretrained=True, res_type=50):
        super(ResNet, self).__init__()

        self.classify = nn.Linear(512, 5)

        pretrained_model = torchvision.models.__dict__['resnet{}'.format(res_type)](pretrained=pretrained)
        self.conv1 = pretrained_model._modules['conv1']
        self.bn1 = pretrained_model._modules['bn1']
        self.relu = pretrained_model._modules['relu']
        self.maxpool = pretrained_model._modules['maxpool']

        self.layer1 = pretrained_model._modules['layer1']
        self.layer2 = pretrained_model._modules['layer2']
        self.layer3 = pretrained_model._modules['layer3']
        self.layer4 = pretrained_model._modules['layer4']

        self.avgpool = nn.AdaptiveAvgPool2d(1)

        del pretrained_model

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classify(x)

        return x

models_18 = [ResNet(pretrained=False, res_type=18), ResNet(pretrained=True, res_type=18)]
models_50 = [ResNet(pretrained=False, res_type=50), ResNet(pretrained=True, res_type=50)]
device = torch.device('cuda')

def train(model):
    model.train()
    correct = 0

    for idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.flatten())
        _, preds = torch.max(outputs, 1) # the second return of max is the return of argmax
        loss.backward()
        optimizer.step()
        correct += torch.sum(preds == labels.data.flatten())
        if idx%1000 == 0:
            print(idx)
            
    epoch_acc = 100*correct.item() / 28099    
    print('Train Acc: {:4f}'.format(epoch_acc))

    return epoch_acc

def test(model, best_acc, best_model_weight):
    # i indicates that which model we are running now
    model.eval()
    correct = 0

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs.float())

        _, preds = torch.max(outputs, 1) # the second return of max is the return of argmax
        correct += torch.sum(preds == labels.data.flatten())
        if idx%1000 == 0:
            print(idx)
            
    epoch_acc = 100*correct.item() / 7025    
    print('Test Acc: {:4f}'.format(epoch_acc))

    if epoch_acc > best_acc:
        best_model_weight = copy.deepcopy(model.state_dict())

    return epoch_acc, best_model_weight

model_weight = []
legend = ['Train(w/o pretrain)', 'Test(w/o pretrain)', 'Test(w/o pretrain)', 'Test(with pretrain)']
for i, model in enumerate(models_18):
    best_acc = 0
    model.to(device)
    train_acc = []
    test_acc = []
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    weight = copy.deepcopy(model.state_dict())

    for epoch in range(epochs_18):
        print('Epoch {}'.format(epoch+1))
        train_acc.append(train(model))
        acc, weight = test(model, best_acc, weight)
        test_acc.append(acc)
        if acc > best_acc:
            best_acc = acc

        print('-' * 10)
    
    model_weight.append(weight)
    print('Best Acc: {:4f}'.format(best_acc))
    plt.plot(range(epochs), train_acc, label=legend[2*i])
    plt.plot(range(epochs), test_acc, label=legend[2*i+1])

plt.xlabel('Epochs')
plt.ylabel('Accuracy(%)')
plt.title("Result comparison(ResNet18)")
plt.legend(loc='best')
plt.savefig("Result_ResNet18.png")
plt.show()

for i, model in enumerate(models_50):
    best_acc = 0
    model.to(device)
    train_acc = []
    test_acc = []
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    weight = copy.deepcopy(model.state_dict())

    for epoch in range(epochs_50):
        print('Epoch {}'.format(epoch+1))
        train_acc.append(train(model))
        acc, weight = test(model, best_acc, weight)
        test_acc.append(acc)
        if acc > best_acc:
            best_acc = acc

        print('-' * 10)
    
    model_weight.append(weight)
    print('Best Acc: {:4f}'.format(best_acc))
    plt.plot(range(epochs), train_acc, label=legend[2*i])
    plt.plot(range(epochs), test_acc, label=legend[2*i+1])

plt.xlabel('Epochs')
plt.ylabel('Accuracy(%)')
plt.title("Result comparison(ResNet50)")
plt.legend(loc='best')
plt.savefig("Result_ResNet50.png")
plt.show()

> Found 28099 images...
> Found 7025 images...
Epoch 1
0
